In [1]:
# Import Python package
import binhosupernova
from binhosupernova.supernova import Supernova
from binhosupernova.commands.system.definitions import GetUsbStringSubCommand
from binhosupernova.commands.i2c.definitions import *

## Getting started

#### 1. List all the Supernova devices connected to the PC host

The ``binhosupernova.getConnectedSupernovaDevicesList()`` gets a list of the Supernova devices plugged into the host PC machine.

In [2]:
binhosupernova.getConnectedSupernovaDevicesList()

[{'path': '\\\\?\\HID#VID_1FC9&PID_82FC#7&9e2d326&0&0000#{4d1e55b2-f16f-11cf-88cb-001111000030}',
  'vendor_id': '0x1fc9',
  'product_id': '0x82fc',
  'serial_number': '5F4BCE79166A34559DF3081DF5A8769D',
  'manufacturer_string': 'Binho LLC',
  'product_string': 'Binho Supernova',
  'hardware_version': 'C',
  'firmware_version': '4.1.2'}]

#### 2. Create an instance of the Supernova class

To utilize a Supernova USB host adapter device, we need to create an instance of the Supernova class.

In [3]:
# Create a device instance. One instance of the Supernova class represents a Supernova USB host adapter device.
supernova = Supernova()

#### 3. Open connection to the Supernova device

The public method ``Supernova.open()`` establishes the connection with a Supernova device. Below is the complete signature:

```python
open(serial, path)
```

- ``serial: str``: The Supernova serial number.
- ``path: str``: The OS HID path assigned to the device. This can be obtained using the ``binhosupernova.getConnectedSupernovaDevicesList()`` method. The ``path`` parameter is currently the only way to uniquely identify each Supernova device. Therefore, it is recommended to use the ``path`` parameter, especially when opening connections with more than one Supernova device simultaneously.

In [4]:
# Use the method by default to connect to only one Supernova device.
supernova.open()

# Otherwise, use the path attribute to identify each Supernova device. Uncomment the line below and comment the line #2.
# supernova.open(path='\\\\?\\HID#VID_1FC9&PID_82FC#6&48d9417&0&0000#{4d1e55b2-f16f-11cf-88cb-001111000030}')

{'module': 0,
 'opcode': 0,
 'message': 'Connection with Supernova device opened successfully.'}

#### 4. Define and register a callback to handle responses and notifications from Supernova

To handle responses and notifications from Supernova, a callback function must be defined and registered. This function will be invoked every time the Supernova sends a response to a request, an asynchronous notification, or a message from the system.

The callback function's signature is as follows: 

``def callback_function_name(supernova_message: dict, system_message: dict) -> None:``

Once the callback function is defined, it should be registered using the ``Supernova.onEvent(callback_function)`` method.

In [5]:
# Define callback function
def callback_function(supernova_message: dict, system_message: dict) -> None:

    if supernova_message != None:

        # Print mesasage
        print(">> New message from SUPERNOVA:")
        print(supernova_message)

    if system_message != None:

        # Print message
        print(">> New message from the SYSTEM:")
        print(system_message)

In [6]:
# Register callback function
supernova.onEvent(callback_function)

{'module': 0,
 'opcode': 0,
 'message': 'On event callback function registered successfully.'}

#### 5. Define a function to generate transaction IDs

All the request messages sent to the Supernova from the USB Host application must include the transaction or request ID. The ID is a 2-byte integer with an allowed range of ``[1, 65535]``.

In this example, a dummy function called ``getId()`` is defined to increment a transaction counter used as the ID.

In [7]:
#Auxiliar code to generate IDs.

counter_id = 0

def getId():
    global counter_id
    counter_id = counter_id + 1
    return counter_id

#### 6. Test communication with a basic command such as ``GET USB STRING``

The ``GET USB STRING`` command allows you to retrieve various string values related to the product, such as the manufacturer, product name, serial number, firmware version, and hardware version. This command serves as a basic test to verify the communication and functionality with the Supernova device. By successfully receiving the expected string values, you can ensure that the communication between the PC host and the Supernova device is functioning as expected.

In [8]:
# GET USB STRING - Get Manufacturer
request_result = supernova.getUsbString(getId(), GetUsbStringSubCommand.MANUFACTURER)

# GET USB STRING  - Get Product Name
request_result = supernova.getUsbString(getId(), GetUsbStringSubCommand.PRODUCT_NAME)

# GET USB STRING  - Get Serial Number
request_result = supernova.getUsbString(getId(), GetUsbStringSubCommand.SERIAL_NUMBER)

# GET USB STRING  - Firmware Version
request_result = supernova.getUsbString(getId(), GetUsbStringSubCommand.FW_VERSION)

# GET USB STRING  - Hardware Version
request_result = supernova.getUsbString(getId(), GetUsbStringSubCommand.HW_VERSION)

>> New message from SUPERNOVA:
{'id': 1, 'command': 'SYS GET USB STRING', 'result': 'SUCCESS', 'payload_length': 12, 'payload': 'MN-Binho LLC'}
>> New message from SUPERNOVA:
{'id': 2, 'command': 'SYS GET USB STRING', 'result': 'SUCCESS', 'payload_length': 18, 'payload': 'PR-Binho Supernova'}
>> New message from SUPERNOVA:
{'id': 3, 'command': 'SYS GET USB STRING', 'result': 'SUCCESS', 'payload_length': 35, 'payload': 'SN-5F4BCE79166A34559DF3081DF5A8769D'}
>> New message from SUPERNOVA:
{'id': 4, 'command': 'SYS GET USB STRING', 'result': 'SUCCESS', 'payload_length': 8, 'payload': 'FW-4.1.2'}
>> New message from SUPERNOVA:
{'id': 5, 'command': 'SYS GET USB STRING', 'result': 'SUCCESS', 'payload_length': 4, 'payload': 'HW-C'}


## I2C Protocol API

The I2C Protocol API methods are described below.

``Supernova.setI2cSpiUartGpioVoltage(id, voltage_mV)``

This method supplies the indicated voltage to the bus in mV, ranging from 1200 mV up to 3300 mV. It also allows setting the value 0 mV to power off the output voltage.

- ``id: int`` : A 2-byte integer that represents the transfer ID.
- ``voltage_mV: c_int16 ``: The voltage parameter is a 2-byte integer in the range [1200, 3300] mV and allows the value 0 mV to power off the output voltage.

In [9]:
supernova.setI2cSpiUartGpioVoltage(getId(), 3300)

{'module': 1,
 'opcode': 0,
 'message': 'SET I2C SPI UART GPIO VOLTAGE request success'}

>> New message from SUPERNOVA:
{'id': 6, 'command': 'SYS SET I2C SPI UART GPIO VOLTAGE', 'result': 'SUCCESS'}


``Supernova.i2cControllerInit(id, frequency, pullUpResistorValue)``

This method initializes the Supernova as an I2C controller.

  - ``id: int`` : A 2-byte integer that represents the transfer ID.
  - ``frequency : int`` : This parameter represents the I2C SCL frequency in Hz. Currently, the minimum allowed value is 100000 Hz and the maximum allowed value is 1000000 Hz.
  - ``pullUpResistorsValue: I2cPullUpResistorsValue`` : This parameter indicates the desire pull up resistors value to be set in the SDA and SCL lines. This parameter can take the following values:

    - ``I2cPullUpResistorsValue.I2C_PULLUP_10kOhm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_4_7kOhm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_3_3kOhm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_2_2kOhm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_1_5kOhm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_1kOhm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_680Ohm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_470Ohm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_330Ohm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_220Ohm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_150Ohm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_DISABLED``

In [10]:
supernova.i2cControllerInit(getId(), 1000000, I2cPullUpResistorsValue.I2C_PULLUP_DISABLE)

{'module': 1, 'opcode': 0, 'message': 'I2C CONTROLLER INIT request success'}

>> New message from SUPERNOVA:
{'id': 7, 'command': 'I2C CONTROLLER INIT', 'result': 'SUCCESS', 'i2c_bus': 'I2C_BUS_A'}


``Supernova.i2cControllerSetParameters(id, frequency, pullUpResistorValue)``

This method sets the I2C transfers frquency and pull up resistors value for the SDA and SCL lines.

  - ``id: int`` : A 2-byte integer that represents the transfer ID.
  - ``frequency : int`` : This parameter represents the I2C SCL frequency in Hz. Currently, the minimum allowed value is 100000 Hz and the maximum allowed value is 1000000 Hz.
  - ``pullUpResistorsValue: I2cPullUpResistorsValue`` : This parameter indicates the desire pull up resistors value to be set in the SDA and SCL lines. This parameter can take the following values:

    - ``I2cPullUpResistorsValue.I2C_PULLUP_10kOhm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_4_7kOhm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_3_3kOhm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_2_2kOhm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_1_5kOhm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_1kOhm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_680Ohm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_470Ohm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_330Ohm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_220Ohm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_150Ohm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_DISABLED``

In [11]:
supernova.i2cControllerSetParameters(getId(), frequency=100000, pullUpResistorsValue=I2cPullUpResistorsValue.I2C_PULLUP_2_2kOhm)

{'module': 1, 'opcode': 0, 'message': 'I2C SET PARAMETERS request success'}

>> New message from SUPERNOVA:
{'id': 8, 'command': 'I2C CONTROLLER SET PARAMETERS', 'result': 'SUCCESS', 'i2c_bus': 'I2C_BUS_A'}


``Supernova.i2cSetPullUpResistors(id, pullUpResistorsValue)``

This method sets the I2C pull up resistors value for the SDA and SCL signals.

  - ``id: int`` : A 2-byte integer that represents the transfer ID.
  - ``pullUpResistorsValue: I2cPullUpResistorsValue`` : This parameter indicates the desire pull up resistors value to be set in the SDA and SCL lines. This parameter can take the following values:

    - ``I2cPullUpResistorsValue.I2C_PULLUP_DISABLE``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_10kOhm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_4_7kOhm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_3_3kOhm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_2_2kOhm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_1_5kOhm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_1kOhm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_680Ohm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_470Ohm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_330Ohm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_220Ohm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_150Ohm``

In [12]:
request_result = supernova.i2cSetPullUpResistors(getId(), I2cPullUpResistorsValue.I2C_PULLUP_4_7kOhm)

>> New message from SUPERNOVA:
{'id': 9, 'command': 'I2C SET PULLUP RESISTORS', 'result': 'SUCCESS', 'i2c_bus': 'I2C_BUS_A'}


``Supernova.i2cControllerScanBus(id, include10BitAddresses = False)``

This method is used to request to the Supernova device to perform an I2C scan bus. The
I2C scan bus sends a ping transfer to all the available addresses and retrieves a list with the detected addresses.

  - ``id: int`` : It is a 2-bytes integer that represent the transfer id.
  - ``include10BitAddresses: bool = False`` : This optional flag indicates that the 10-bit addresses must be included in the scan process.

In [13]:
supernova.i2cControllerScanBus(getId())

{'module': 1,
 'opcode': 0,
 'message': 'I2C CONTROLLER SCAN BUS request success'}

>> New message from SUPERNOVA:
{'id': 10, 'command': 'I2C CONTROLLER SCAN BUS', 'result': 'SUCCESS', 'i2c_bus': 'I2C_BUS_A', 'detected_7_bit_addresses': [80], 'detected_10_bit_addresses': []}


``Supernova.i2cControllerWrite(id, targetAddress, registerAddress: list, data: list, isNonStop: bool = False)``

This method is used to request to the the Supernova device to perform an I2C write transfer. The
I2C write transfer starts with a START condition and ends with a STOP condition.

  - ``id: int`` : It is a 2-bytes integer that represent the transfer id.
  - ``targetAddress: c_uint8`` : I2C target static address.
  - ``registerAddress: list `` : A Python list that contains the memory/register address of the I2C target's internal memory, to which the data will be written. The list holds bytes, and can have a length from 0 bytes up to 4 bytes. If the list is empty, the Supernova will ignore it and write only the data payload.
  - ``data: list`` : A Python list that contains the I2C data to be transferred in the I2C write operation. The list holds bytes elements, and the maximum length is 1024 bytes.
  - ``isNonStop: bool`` : (Optional) This parameter indicates if the I2C write transfer is non-stop, meaning that it ends without a STOP condition. If the flag is set to False, the transfer ends with STOP; otherwise, if it's True, the transfer is non-stop, without a STOP condition at the end. By default, the flag is set to False indicating the transfer ends with a STOP condition

In [14]:
TARGET_ADDRESS   = 0x50
TARGET_REGISTER_ADDRESS = [0,0]   # [reg, page]
data = [30 for i in range(1,114)]

request_result = supernova.i2cControllerWrite(getId(), TARGET_ADDRESS, TARGET_REGISTER_ADDRESS, data)

>> New message from SUPERNOVA:
{'id': 11, 'command': 'I2C CONTROLLER WRITE', 'result': 'SUCCESS', 'i2c_bus': 'I2C_BUS_A', 'payload_length': 113}


``Supernova.i2cControllerRead(id, targetAddress, requestDataLength, registerAddress: list = [])``

This method is used to request the USB device to perform an I2C read operation. If the parameter `registerAddress` is populated, the read operation is preceded by an I2C write transfer to send the register/memory address from which the data will be read. A R-Start condition is performed between the I2C write and I2C read transfers.

  - ``id: int`` : A 2-byte integer that represent the transaction ID.
  - ``targetAddress: c_uint8`` : I2C target static address.
  - ``requestDataLength: int`` : Length of the data to be read. The maximum value is 1024 bytes.
  - ``registerAddress: list `` : (Optional) A Python list that contains the memory/register address of the I2C target's internal memory, from which the data will be read. The list holds bytes, and it can have a length from 0 bytes up to 4 bytes. If the list is empty, the Supernova will perform an I2C Read operation without indicating the register address and starts reading from the last position of the previous transfer.

In [15]:
TARGET_ADDRESS =  0x50
READ_LENGTH = 30
# Indicate the register address to perform an I2C Read From operation
TARGET_REGISTER_ADDRESS = [0,0]   # [reg, page]

request_result = supernova.i2cControllerRead(getId(), TARGET_ADDRESS, READ_LENGTH, TARGET_REGISTER_ADDRESS)

>> New message from SUPERNOVA:
{'id': 12, 'command': 'I2C CONTROLLER READ', 'result': 'SUCCESS', 'i2c_bus': 'I2C_BUS_A', 'payload_length': 30, 'payload': [30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30]}


``Supernova.i2cControllerWrite(id, targetAddress, registerAddress: list, data: list, isNonStop: bool = False)``

In this case, the I2C controller write method is used to request the the USB device to perform an I2C write operation that starts with a START condition but the STOP condition is not performed after the last byte. To indicate this, we will set the `isNonStop` flag to True.

In [16]:
TARGET_ADDRESS = 0x50
TARGET_REGISTER_ADDRESS = [0,0]   # [reg, page]
data = [55 for i in range(1,129)]

request_result = supernova.i2cControllerWrite(getId(), TARGET_ADDRESS, TARGET_REGISTER_ADDRESS, data, isNonStop=True)

>> New message from SUPERNOVA:
{'id': 13, 'command': 'I2C CONTROLLER WRITE', 'result': 'SUCCESS', 'i2c_bus': 'I2C_BUS_A', 'payload_length': 128}


``Supernova.i2cControllerRead(id, targetAddress, requestDataLength, registerAddress: list = [])``

To check the correct execution of the ``i2cControllerWrite`` command with non-stop, an ``i2cControllerRead`` command is performed, reading 70 bytes from the slave register [0, 0]

In [17]:
TARGET_ADDRESS =  0x50
READ_LENGTH = 70
TARGET_REGISTER_ADDRESS = [0,0]   # [reg, page]

request_result = supernova.i2cControllerRead(getId(), TARGET_ADDRESS, READ_LENGTH, TARGET_REGISTER_ADDRESS)

>> New message from SUPERNOVA:
{'id': 14, 'command': 'I2C CONTROLLER READ', 'result': 'SUCCESS', 'i2c_bus': 'I2C_BUS_A', 'payload_length': 70, 'payload': [55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55]}


``Supernova.i2cControllerRead(id, targetAddress, requestDataLength, registerAddress: list = [])``

In this case, the I2C controller read method is used to request the USB device to perform an I2C read operation that starts reading at the final position of the previous transfer.

In [18]:
TARGET_ADDRESS =  0x50
READ_LENGTH = 70

request_result = supernova.i2cControllerRead(getId(), TARGET_ADDRESS, READ_LENGTH)

>> New message from SUPERNOVA:
{'id': 15, 'command': 'I2C CONTROLLER READ', 'result': 'SUCCESS', 'i2c_bus': 'I2C_BUS_A', 'payload_length': 70, 'payload': [55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 6, 19, 231, 215, 211, 188, 198, 225, 223, 114, 205, 49]}


## Close communication

Use the ``Supernova.close()`` method to end the communication with the Supernova device and release the used memory in the background like threads and so on.

In [19]:
# Close the communication with the Supernova device.
supernova.close()

{'module': 0, 'opcode': 0, 'message': 'Communication closed successfully.'}